# 🤖 Easy RAG System for Google Colab

This notebook creates a simple but powerful Question Answering system using Retrieval-Augmented Generation (RAG).

**What you'll learn:**
- How RAG systems work
- Building a document knowledge base
- Semantic search with vector embeddings
- Generating contextual answers

**No complex setup required - everything runs in Colab!**

In [ ]:
# Install required packages
!pip install -q faiss-cpu sentence-transformers gradio

In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import gradio as gr
from typing import List, Dict
import warnings
warnings.filterwarnings("ignore")

print("✅ Libraries imported successfully!")

## 🏗️ Building the RAG System

Our RAG system has 3 main components:
1. **Document Store**: Where we keep our knowledge base
2. **Retriever**: Finds relevant documents for a question
3. **Generator**: Creates answers using retrieved context

In [ ]:
class EasyRAG:
    """Simple RAG system for beginners"""
    
    def __init__(self):
        print("🚀 Initializing RAG System...")
        
        # Load embedding model
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        print("📊 Embedding model loaded")
        
        # Create vector database (FAISS)
        self.embedding_dim = 384  # Dimension for all-MiniLM-L6-v2
        self.index = faiss.IndexFlatL2(self.embedding_dim)
        print("🗃️ Vector database initialized")
        
        # Storage for documents
        self.documents = []
        
        print("✅ RAG System ready!")
    
    def add_documents(self, texts: List[str]):
        """Add documents to the knowledge base"""
        print(f"📚 Adding {len(texts)} documents...")
        
        # Create embeddings for all documents
        embeddings = self.embedder.encode(texts, show_progress_bar=True)
        
        # Add embeddings to FAISS index
        self.index.add(embeddings.astype('float32'))
        
        # Store the original texts
        self.documents.extend(texts)
        
        print(f"✅ Total documents in knowledge base: {len(self.documents)}")
    
    def search(self, query: str, top_k: int = 3) -> List[Dict]:
        """Search for relevant documents"""
        if not self.documents:
            return []
        
        # Convert query to embedding
        query_embedding = self.embedder.encode([query])
        
        # Search in FAISS index
        distances, indices = self.index.search(
            query_embedding.astype('float32'), 
            min(top_k, len(self.documents))
        )
        
        # Prepare results
        results = []
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if idx < len(self.documents):
                similarity = 1 / (1 + distance)  # Convert distance to similarity
                results.append({
                    "text": self.documents[idx],
                    "similarity": similarity,
                    "rank": i + 1
                })
        
        return results
    
    def answer_question(self, question: str) -> tuple:
        """Answer a question using retrieval-augmented generation"""
        # Step 1: Retrieve relevant documents
        relevant_docs = self.search(question, top_k=3)
        
        if not relevant_docs:
            return "I don't have information to answer this question.", []
        
        # Step 2: Create context from retrieved documents
        context = "\n\n".join([doc["text"] for doc in relevant_docs])
        
        # Step 3: Generate answer (simple extractive approach)
        # For this simple version, we'll return the most relevant context
        answer = f"Based on the available information:\n\n{relevant_docs[0]['text']}"
        
        if len(relevant_docs) > 1:
            answer += f"\n\nAdditional relevant information:\n{relevant_docs[1]['text'][:200]}..."
        
        return answer, relevant_docs

print("🎯 RAG system class defined!")

## 📚 Creating a Sample Knowledge Base

In [ ]:
# Sample documents for our knowledge base
sample_documents = [
    "Artificial Intelligence (AI) is the simulation of human intelligence in machines that are programmed to think and learn like humans. It includes machine learning, natural language processing, and computer vision.",
    
    "Machine Learning is a subset of AI that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. It focuses on developing computer programs that can access data and use it to learn.",
    
    "Deep Learning is a subset of machine learning that uses neural networks with multiple layers to model and understand complex patterns in data. It has been particularly successful in image recognition and natural language processing.",
    
    "Python is a popular programming language for AI and machine learning due to its simplicity, extensive libraries like TensorFlow, PyTorch, and scikit-learn, and strong community support.",
    
    "Google Colab provides free access to computing resources including GPUs, making it an excellent platform for machine learning experiments and learning AI concepts without expensive hardware.",
    
    "RAG (Retrieval-Augmented Generation) systems combine information retrieval with text generation to produce more accurate and contextual responses by first finding relevant information from a knowledge base.",
    
    "Vector databases store high-dimensional vectors (embeddings) and provide fast similarity search capabilities, which are essential for modern AI applications like recommendation systems and semantic search.",
    
    "FAISS (Facebook AI Similarity Search) is a library for efficient similarity search and clustering of dense vectors, developed by Facebook Research for handling large-scale vector databases.",
    
    "Sentence Transformers generate dense vector representations of sentences that capture their semantic meaning, enabling comparison of text similarity beyond simple keyword matching.",
    
    "Natural Language Processing (NLP) is a branch of AI that helps computers understand, interpret, and manipulate human language, including tasks like translation, sentiment analysis, and question answering."
]

print(f"📖 Created {len(sample_documents)} sample documents")
print("\nFirst document preview:")
print(sample_documents[0])

## 🎬 Initialize and Test the RAG System

In [ ]:
# Create our RAG system
rag = EasyRAG()

# Add documents to the knowledge base
rag.add_documents(sample_documents)

In [ ]:
# Test the system with a sample question
test_question = "What is machine learning?"
print(f"🤔 Question: {test_question}")

answer, retrieved_docs = rag.answer_question(test_question)

print(f"\n🤖 Answer:\n{answer}")

print(f"\n🔍 Retrieved {len(retrieved_docs)} relevant documents:")
for i, doc in enumerate(retrieved_docs):
    print(f"{i+1}. Similarity: {doc['similarity']:.3f}")
    print(f"   Text: {doc['text'][:100]}...\n")

## 🎨 Interactive Web Interface

In [ ]:
def chat_interface(question):
    """Gradio interface function"""
    if not question.strip():
        return "Please enter a question!", "No documents retrieved."
    
    answer, docs = rag.answer_question(question)
    
    # Format retrieved documents for display
    docs_display = "\n\n".join([
        f"📄 Document {i+1} (Similarity: {doc['similarity']:.3f}):\n{doc['text']}"
        for i, doc in enumerate(docs)
    ])
    
    return answer, docs_display

# Create Gradio interface
demo = gr.Interface(
    fn=chat_interface,
    inputs=gr.Textbox(
        label="Ask a Question",
        placeholder="What would you like to know?",
        lines=2
    ),
    outputs=[
        gr.Textbox(label="Answer", lines=6),
        gr.Textbox(label="Retrieved Documents", lines=8)
    ],
    title="🤖 Easy RAG System Demo",
    description="Ask questions about AI, machine learning, and related topics!",
    examples=[
        "What is artificial intelligence?",
        "How does deep learning work?",
        "What are the benefits of Python for AI?",
        "Tell me about Google Colab",
        "What is a vector database?"
    ]
)

# Launch the interface
demo.launch(share=True)

## 🔧 Advanced Features

In [ ]:
# Function to add custom documents
def add_custom_documents(text_input):
    """Add custom text to the knowledge base"""
    if not text_input.strip():
        return "Please provide some text to add."
    
    # Split by double newlines to separate paragraphs
    paragraphs = [p.strip() for p in text_input.split('\n\n') if p.strip()]
    
    if paragraphs:
        rag.add_documents(paragraphs)
        return f"✅ Added {len(paragraphs)} new document(s) to the knowledge base!"
    else:
        return "No valid paragraphs found in the input."

# Create interface for adding documents
add_docs_demo = gr.Interface(
    fn=add_custom_documents,
    inputs=gr.Textbox(
        label="Add New Documents",
        placeholder="Paste your text here. Separate different documents with double newlines.",
        lines=5
    ),
    outputs=gr.Textbox(label="Status"),
    title="📚 Add Documents to Knowledge Base",
    description="Expand the knowledge base by adding your own documents!"
)

add_docs_demo.launch(share=True)

## 🎯 More Examples and Testing

In [ ]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and machine learning?",
    "How does RAG work?",
    "Why is Python popular for AI?",
    "What can I do with Google Colab?"
]

print("🧪 Testing with multiple questions:\n")

for i, question in enumerate(test_questions, 1):
    print(f"Question {i}: {question}")
    answer, docs = rag.answer_question(question)
    print(f"Answer: {answer[:200]}...")
    print(f"Retrieved {len(docs)} documents\n")
    print("-" * 50)

## 📊 System Statistics

In [ ]:
# Display system information
print("📊 RAG System Statistics:")
print(f"📚 Total documents: {len(rag.documents)}")
print(f"🧮 Embedding dimension: {rag.embedding_dim}")
print(f"🗃️ FAISS index size: {rag.index.ntotal}")
print(f"🔢 Average document length: {np.mean([len(doc) for doc in rag.documents]):.0f} characters")

# Show sample of document lengths
doc_lengths = [len(doc) for doc in rag.documents]
print(f"📏 Document length range: {min(doc_lengths)} - {max(doc_lengths)} characters")

## 🎉 Congratulations!

You've successfully built and tested a RAG system! Here's what you've accomplished:

✅ **Built a complete RAG pipeline** with document storage, retrieval, and generation

✅ **Used semantic search** with vector embeddings to find relevant information

✅ **Created an interactive web interface** for easy testing

✅ **Learned to add custom documents** to expand the knowledge base

## 🚀 Next Steps

You can enhance this system by:

1. **Adding more documents** - Upload text files or paste content
2. **Using better language models** - Integrate OpenAI API or local models like Ollama
3. **Improving chunking** - Split long documents into smaller, more focused pieces
4. **Adding metadata** - Include categories, dates, or sources for better filtering
5. **Implementing re-ranking** - Use additional models to improve retrieval quality

## 📚 Resources for Learning More

- [LangChain Documentation](https://docs.langchain.com/)
- [Sentence Transformers](https://www.sbert.net/)
- [FAISS Documentation](https://faiss.ai/)
- [RAG Paper](https://arxiv.org/abs/2005.11401)

Happy building! 🤖✨